In [1]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim

In [2]:
# reading the data and checking the content
df = pd.read_csv('data_files/default_data.csv')
print(df)
print(df.keys())
print(len(df))

           X1  X2  X3  X4  X5  X6  X7  X8      X9     X10  ...    X12    X13  \
0       20000  24   2   2  -1  -1  -2  -2    3913    3102  ...      0      0   
1      120000  26  -1   2   0   0   0   2    2682    1725  ...   3272   3455   
2       90000  34   0   0   0   0   0   0   29239   14027  ...  14331  14948   
3       50000  37   0   0   0   0   0   0   46990   48233  ...  28314  28959   
4       50000  57  -1   0  -1   0   0   0    8617    5670  ...  20940  19146   
...       ...  ..  ..  ..  ..  ..  ..  ..     ...     ...  ...    ...    ...   
29995  220000  39   0   0   0   0   0   0  188948  192815  ...  88004  31237   
29996  150000  43  -1  -1  -1  -1   0   0    1683    1828  ...   8979   5190   
29997   30000  37   4   3   2  -1   0   0    3565    3356  ...  20878  20582   
29998   80000  41   1  -1   0   0   0  -1   -1645   78379  ...  52774  11855   
29999   50000  46   0   0   0   0   0   0   47929   48905  ...  36535  32428   

         X14    X15    X16    X17   X18

In [3]:
# loading the data values and target values
X = df[[f'X{i}' for i in range(1, 21)]].to_numpy()
Y = df['Y'].to_numpy()

In [4]:
# creating the class
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.net = nn.Sequential(nn.Linear(20, 10, bias=False), nn.Sigmoid(),
                                 nn.Linear(10, 1, bias=False), nn.Sigmoid())
        
    def forward(self, x):
        return self.net(x)

In [5]:
# train-test-val split (non-random, don't do that in practice)
x_train = torch.from_numpy(X[:25000, :]).float()
y_train = torch.from_numpy(Y[:25000]).float()
x_val = torch.from_numpy(X[25000:27500, :]).float()
y_val = torch.from_numpy(Y[25000:27500]).float()
x_test = torch.from_numpy(X[27500:, :]).float()
y_test = torch.from_numpy(Y[27500:]).float()

In [6]:
# creating an instance of the the network, optimizer and loss function
net = Net()
optimizer = optim.SGD(net.parameters(), lr=0.1)
criterion = nn.BCELoss()

In [7]:
# training
for epoch in range(100):
    optimizer.zero_grad()
    y = net(x_train)
    loss = criterion(y, y_train)
    loss.backward()
    optimizer.step()
    print('{}, {}'.format(epoch, loss))

/Users/mbolonkin/opt/anaconda3/lib/python3.7/site-packages/torch/nn/modules/loss.py:498: UserWarning: Using a target size (torch.Size([25000])) that is different to the input size (torch.Size([25000, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


0, 0.6676478981971741
1, 0.6334870457649231
2, 0.6032590270042419
3, 0.5661739706993103
4, 0.5535538792610168
5, 0.5490840077400208
6, 0.5407330989837646
7, 0.5392913818359375
8, 0.5367408394813538
9, 0.5355471968650818
10, 0.5335676074028015
11, 0.5343053340911865
12, 0.5324618816375732
13, 0.5317046642303467
14, 0.5303352475166321
15, 0.5299214124679565
16, 0.5300446152687073
17, 0.5293494462966919
18, 0.5291922092437744
19, 0.5291098952293396
20, 0.5289309620857239
21, 0.5288582444190979
22, 0.5285044312477112
23, 0.5284377932548523
24, 0.5288742780685425
25, 0.5285923480987549
26, 0.5285981297492981
27, 0.528698205947876
28, 0.5282694697380066
29, 0.5281371474266052
30, 0.5282219648361206
31, 0.528232991695404
32, 0.5282230973243713
33, 0.5316749215126038
34, 0.5316006541252136
35, 0.5313166379928589
36, 0.5312237739562988
37, 0.5307661890983582
38, 0.5318647623062134
39, 0.5316625237464905
40, 0.5317936539649963
41, 0.531665027141571
42, 0.5317766666412354
43, 0.5315932631492615
4

In [8]:
y_pred = net(x_val)
y_pred = (y_pred>=0.5).int()
print((y_pred[:,0]==y_val).float().mean())

tensor(0.7936)
